In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "/content/drive/MYDrive/glove"
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d "/content/drive/MyDrive/glove"

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [ ]:
!pip install OpenNMT-py==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 6.9 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 57 kB 6.2 MB/s 
     |████████████████████████████████| 16.9 MB 13.6 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1


In [ ]:
!pip install -r /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/requirements/requirements1.2.0.opt.txt

In [ ]:
!pip install OpenNMT-py

In [ ]:
!onmt_preprocess \
-train_src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/src_train.txt \
-train_tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/tgt_train.txt \
-valid_src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/src_dev.txt \
-valid_tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/tgt_dev.txt \
-save_data /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/demo

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/tools/embeddings_to_torch.py \
-emb_file_both /content/drive/MyDrive/glove/glove.6B.100d.txt \
-dict_file /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/demo.vocab.pt \
-output_file /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/embeddings

In [ ]:
!onmt_train -save_model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/model1 \
           -batch_size 64 \
           -layers 2 \
           -rnn_size 500 \
           -word_vec_size 500 \
           -pre_word_vecs_enc /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/embeddings.enc.pt \
           -pre_word_vecs_dec /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/embeddings.dec.pt \
           -data /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/demo \
           -world_size 1 -gpu_ranks 0 \
           -save_checkpoint_steps 10000 \
           -report_every 5000  


[2022-06-28 04:39:34,113 INFO]  * src vocab size = 5897
[2022-06-28 04:39:34,113 INFO]  * tgt vocab size = 5562
[2022-06-28 04:39:34,113 INFO] Building model...
[2022-06-28 04:39:38,165 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(5897, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(5562, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (li

In [ ]:
!onmt_translate -model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/model1_step_70000.pt \
-src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/src_test.txt \
 -tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/tgt_test.txt \
  -output /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_test_70000.txt

[2022-06-28 10:11:16,763 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-06-28 10:12:50,778 INFO] PRED AVG SCORE: -0.1347, PRED PPL: 1.1441
[2022-06-28 10:12:50,778 INFO] GOLD AVG SCORE: -19.7539, GOLD PPL: 379340673.4695


In [ ]:
!onmt_translate -model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/model1_step_70000.pt \
-src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/src_dev.txt \
 -tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/tgt_dev.txt \
  -output /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_dev_70000.txt

[2022-07-08 12:27:57,673 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-07-08 12:28:52,380 INFO] PRED AVG SCORE: -0.1234, PRED PPL: 1.1313
[2022-07-08 12:28:52,380 INFO] GOLD AVG SCORE: -11.3513, GOLD PPL: 85074.4792


In [ ]:
!onmt_translate -model /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/model1_step_70000.pt \
-src /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/src_train.txt \
 -tgt /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/tgt_train.txt \
  -output /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_train_70000.txt

[2022-07-08 06:56:46,354 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-07-08 07:03:28,565 INFO] PRED AVG SCORE: -0.0012, PRED PPL: 1.0012
[2022-07-08 07:03:28,565 INFO] GOLD AVG SCORE: -0.1663, GOLD PPL: 1.1809


In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/evaluation.py \
--gold /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files1/dev_gold.txt \
--pred /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_dev_70000.txt --etype all \
--db /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/database \
--table /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/tables.json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
eval_err_num:1
easy pred: SELECT count(*) FROM artist
easy gold: SELECT count(*) FROM singer

eval_err_num:2
easy pred: SELECT count(*) FROM enzyme
easy gold: SELECT count(*) FROM singer

eval_err_num:3
medium pred: SELECT name , country , age FROM artist ORDER BY age DESC
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

eval_err_num:4
medium pred: SELECT name , name , age FROM Person ORDER BY age DESC
medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC

eval_err_num:5
medium pred: SELECT avg(age) , avg(age) , max(age) FROM artist WHERE country = 'United States'
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

eval_err_num:6
medium pred: SELECT avg(age) , avg(age) , max(age) FROM weather
medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'

eval_err_num:7

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/evaluation.py \
--gold /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files1/test_gold.txt \
--pred /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_test_70000.txt \
--etype all --db /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/database \
--table /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/tables.json

Streaming output truncated to the last 5000 lines.

eval_err_num:418
hard pred: SELECT state_province_county FROM addresses WHERE line_1_number_building = (SELECT county_ID FROM college)
hard gold: SELECT border FROM border_info WHERE state_name  =  ( SELECT state_name FROM state WHERE population  =  ( SELECT MAX ( population ) FROM state ) );

eval_err_num:419
hard pred: SELECT state_province_county FROM addresses ORDER BY enr LIMIT 1
hard gold: SELECT border FROM border_info WHERE state_name  =  ( SELECT state_name FROM state WHERE area  =  ( SELECT MIN ( area ) FROM state ) );

eval_err_num:420
hard pred: SELECT state_province_county FROM addresses WHERE state = 'History'
hard gold: SELECT border FROM border_info WHERE state_name  =  ( SELECT state_name FROM state WHERE area  =  ( SELECT MIN ( area ) FROM state ) );

eval_err_num:421
easy pred: SELECT name FROM address WHERE major = 'Public'
easy gold: SELECT traverse FROM river WHERE LENGTH  >  750;

eval_err_num:422
easy pred: SEL

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/evaluation.py \
--gold /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files1/train_gold.txt \
--pred /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/text_files/pred_train_70000.txt \
--etype all --db /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/database \
--table /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/tables.json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
eval_err_num:1
extra pred: SELECT T1.state FROM customer_addresses AS T1 JOIN department AS T2 ON T1.head = T2.organization_id WHERE dept_name = "Brazil"
extra gold: SELECT T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Treasury' INTERSECT SELECT T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Homeland Security'

eval_err_num:2
easy pred: SELECT count(*) FROM Employees;
easy gold: SELECT count(*) FROM Employee

eval_err_num:3
extra pred: SELECT T2.name FROM GENRE AS T1 JOIN playlists AS T2 ON T1.bandmate = T2.id JOIN Songs AS T3 ON T3.id = T2.media_type_id WHERE T3.name = "Rock" AND T3.name != "MPEG audio file";
extra gold: SELECT T1.name FROM tracks

In [ ]:
!python /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/evaluation.py \
--gold /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/my_text_files/my_train_gold_final.txt \
--pred /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/my_text_files/my_pred_train_final.txt \
--etype all --db /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/database \
--table /content/drive/MyDrive/Text_to_SQL_using_OpenNMT_and_Wiki_Spider_data/spider/spider_data/db_and_json_files/tables.json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
                     easy                 medium               hard                 extra                all                 
count                1428                 2290                 1258                 815                  5791                
=====================   EXECUTION ACCURACY     =====================
execution            1.000                1.000                1.000                0.998                1.000               

====================== EXACT MATCHING ACCURACY =====================
exact match          1.000                1.000                1.000                1.000                1.000               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               1.000                1.000                1.000                1.000                1.000               
select(no AGG)       1.000                1.000        